In [2]:
import scipy                        
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy
import pandas
import sklearn
from sklearn import preprocessing
from sklearn import linear_model
import tensorflow

In [4]:
matlabData = sio.loadmat(file_name='data/s2_sl2p_weiss_or_prosail_inout.mat', variable_names=['Input', 'Output'])

In [5]:
                                                                # B1  443nm, Ultra blue, 60m res
                                                                # B2  490nm, blue, 10m res
B3 = pandas.DataFrame(matlabData['Input']['Rho_Toc'][0][0])[0]      # 560nm, green, 10m res
B4 = pandas.DataFrame(matlabData['Input']['Rho_Toc'][0][0])[1]      # 665nm, red, 10m res
B5 = pandas.DataFrame(matlabData['Input']['Rho_Toc'][0][0])[2]      # 705nm, VNIR, 20m res
B6 = pandas.DataFrame(matlabData['Input']['Rho_Toc'][0][0])[3]      # 740nm, VNIR, 20m res
B7 = pandas.DataFrame(matlabData['Input']['Rho_Toc'][0][0])[4]      # 783nm, VNIR, 20m res
                                                                # B8  842nm, VNIR, 10m res
B8A = pandas.DataFrame(matlabData['Input']['Rho_Toc'][0][0])[5]     # 865nm, VNIR, 20m res
                                                                # B9  940nm, SWIR, 60m res
                                                                # B10 1375nm, SWIR, 60m res
B11 = pandas.DataFrame(matlabData['Input']['Rho_Toc'][0][0])[6]     # 1610nm, SWIR, 20m res
B12 = pandas.DataFrame(matlabData['Input']['Rho_Toc'][0][0])[7]     # 2190nm, SWIR, 20m res

LAI = pandas.Series(data=matlabData['Output']['LAI'][0][0].flatten())
FAPAR = pandas.Series(data=matlabData['Output']['FAPAR'][0][0].flatten())
FCOVER = pandas.Series(data=matlabData['Output']['FCOVER'][0][0].flatten())

LAI_mean = LAI.mean()
FAPAR_mean = FAPAR.mean()
FCOVER_mean = FCOVER.mean()

In [6]:
RVI1    = B4/B8A
RVI2    = B4/B7
RVI3    = B4/B6
RVI4    = B4/B5
GM1     = B6/B3
GM2     = B6/B5
GI      = B3/B4
II      = B11/B12
MSI     = B11/B7
PSSR    = B7/B4
SGI     = B8A/B4
SR2     = B7/B3
SR3     = B5/B4
GVI     = (B8A/B3)-1
PSRI    = (B4-B3)/B6
NDVI3   = ((B8A-B4)/(B8A))+B4
SR5     = B4/(B5*B4)
SR6     = B4/(B3*B5)
SR7     = B8A/(B3*B5)
IPVI    = B7/(B7+B4)
ARI     = (1/B3)-(1/B5)
ARI2    = B7*((1/B3)-(1/B5))
NDVI    = (B8A-B4)/(B8A+B4)
GNDVI   = (B8A-B3)/(B8A+B3)
NDVI3   = ((B8A-B4)/(B8A))+B4
NDWI    = (B8A-B11)/(B8A+B11)
NDREVI  = (B8A-B5)/(B8A+B5)
NDGI    = (B3-B4)/(B3+B4)
NDI1    = (B7-B5)/(B7-B4)
NDI2    = (B8A-B5)/(B8A-B4)
RENDVI  = (B6-B5)/(B6+B5)
OSAVI   = (1.16*(B7-B4))/(B7+B4+0.61)
NMDI    = (B8A-(B11-B12))/(B8A+(B11-B12))
HI      = ((B3-B5)/(B3+B5))-0.5*B5
GVSP    = (-0.283*B3 - 0.66*B4 + 0.577*B6 + 0.388*B8A)/(0.433*B3 - 0.632*B4 + 0.586*B6 + 0.264*B8A)
MCARI   = ((B5-B4)-0.2*(B5-B3))*(B5/B4)
TCARI   = 3*((B5-B4)-0.2*(B5-B3)*(B5/B4))
EVI     = 2.5*((B8A-B4)/(B8A+6*B4-7.5*B3+1))
EVI2    = 2.5*((B8A-B4)/(B8A+2.4*B4+1))
RDVI    = (B8A-B4)/((B8A+B4)**0.5)
MSR     = ((B8A/B4)-1)/((B8A/B4)**0.5+1)
MSAVI   = 0.5*(2*B7+1-((2*B7+1)**2-8*(B7-B4))**0.5)
MSAVI2  = 0.5*(2*B8A+1-((2*B8A+1)**2-8*(B8A-B4))**0.5)
MCARI2  = (1.5*(2.5*(B7-B4)-1.3*(B7-B3)))/((((2*B7+1)**2)-(6*B7-5*(B4**0.5))-0.5)**0.5)
MTVI2   = (1.5*(1.2*(B7-B3)-2.5*(B4-B3)))/(((2*B7+1)**2-(6*B7-5*B4)-0.5)**0.5)
N       = (2*((B8A**2)-(B4**2))+1.5*B8A+0.5*B4)/(B8A+B4+0.5)
GEMI    = N*(1-0.25*N)-((B4-0.125)/(1-B4))
MSR2    = ((B7/B4)-1)/(((B7/B4)+1)**0.5)
NLI     = ((B8A**2)-B4)/((B8A**2)+B4)
TDVI    = (0.5+((B8A-B4)/(B8A+B4)))**0.5

add correlation matrix calculation, and single feature collection

In [7]:
outputCSV = pandas.concat([ B3, B4, B5, B6, B7, B8A, B11, B12,
                            RVI1, RVI2, RVI3, RVI4, GM1, GM2, GI, II, MSI,
                            PSSR, SGI, SR2, SR3, GVI, PSRI, NDVI3, SR5, SR6,
                            SR7, IPVI, ARI, ARI2, NDVI, GNDVI, NDWI, NDREVI, NDGI,
                            NDI1, NDI2, RENDVI, OSAVI, NMDI, HI, GVSP, MCARI,
                            TCARI, EVI, EVI2, RDVI, MSR, MSAVI, MSAVI2, MCARI2,
                            MTVI2, N, GEMI, MSR2, NLI, TDVI, LAI, FAPAR, FCOVER ], 
                            axis=1, join='outer')
outputCSV['latitude'] = 0
outputCSV['longitude'] = 0
outputCSV.columns =['B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12',
                    'RVI1', 'RVI2', 'RVI3', 'RVI4', 'GM1', 'GM2', 'GI', 'II', 'MSI',
                    'PSSR', 'SGI', 'SR2', 'SR3', 'GVI', 'PSRI', 'NDVI3', 'SR5', 'SR6',
                    'SR7', 'IPVI', 'ARI', 'ARI2', 'NDVI', 'GNDVI', 'NDWI', 'NDREVI', 'NDGI',
                    'NDI1', 'NDI2', 'RENDVI', 'OSAVI', 'NMDI', 'HI', 'GVSP', 'MCARI',
                    'TCARI', 'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI', 'MSAVI2', 'MCARI2',
                    'MTVI2', 'N', 'GEMI', 'MSR2', 'NLI', 'TDVI', 'LAI', 'FAPAR', 'FCOVER', 
                    'latitude', 'longitude']
outputCSV

,B3,B4,B5,B6,B7,B8A,B11,B12,RVI1,RVI2,...,N,GEMI,MSR2,NLI,TDVI,LAI,FAPAR,FCOVER,latitude,longitude
0,0.048953,0.030271,0.073742,0.294630,0.419698,0.436037,0.121603,0.043615,0.069422,0.072125,...,1.084141,0.887987,3.336756,0.725310,1.170542,6.014594,0.870412,0.875424,0,0
1,0.053655,0.029552,0.074523,0.383288,0.566787,0.597187,0.180981,0.054689,0.049485,0.052139,...,1.439619,1.019848,4.046955,0.846956,1.185621,5.752335,0.957268,0.971936,0,0
2,0.075767,0.092849,0.144840,0.267414,0.306598,0.321708,0.254556,0.145838,0.288614,0.302837,...,0.785885,0.666923,1.109901,0.054225,1.025698,0.746137,0.386350,0.394260,0,0
3,0.071742,0.031115,0.094643,0.303518,0.383710,0.391068,0.118468,0.047038,0.079565,0.081091,...,0.982550,0.838099,3.103530,0.661880,1.163012,5.374681,0.819419,0.830017,0,0
4,0.100043,0.028230,0.130998,0.442701,0.560120,0.563746,0.182785,0.062757,0.050075,0.050399,...,1.367942,0.999707,4.127172,0.836842,1.185169,5.467322,0.899630,0.927706,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41467,0.091781,0.028950,0.124602,0.420813,0.528081,0.544847,0.122709,0.032790,0.053135,0.054822,...,1.325936,0.985322,3.930491,0.822286,1.182832,4.951841,0.922245,0.949148,0,0
41468,0.046467,0.034942,0.069040,0.229278,0.308859,0.321834,0.126807,0.071680,0.108572,0.113133,...,0.822777,0.746855,2.499143,0.495492,1.141982,4.453112,0.717015,0.714964,0,0
41469,0.045621,0.027923,0.067862,0.307144,0.445989,0.474847,0.162763,0.055750,0.058805,0.062610,...,1.172384,0.928628,3.634234,0.779614,1.178526,4.579922,0.861480,0.871602,0,0
41470,0.053039,0.037354,0.081990,0.340293,0.506438,0.528001,0.190817,0.076028,0.070745,0.073757,...,1.281693,0.962056,3.291311,0.763689,1.169555,5.502935,0.879030,0.882815,0,0


In [26]:
outputCSV.to_csv(path_or_buf=r'transformedFeatures.csv')

In [8]:
import ee
ee.Initialize()

In [9]:
def EE_LARS_Regression(assetName, features, response, maxSamples, n_nonzero):
        
    inputCSV = ee.FeatureCollection(assetName)
    inputCSV = inputCSV.toList(count=maxSamples)
    
    def extractBands(feature):
        feature = ee.Feature(feature)
        return feature.toArray(properties=features).toList()

    def extractVI(feature):
        feature = ee.Feature(feature)
        return feature.toArray(properties=[response]).toList()
    
    inputList = inputCSV.map(extractBands)
    outputList = inputCSV.map(extractVI)
    
    X = ee.Array(inputList)
    y = ee.Array(outputList)
    
    n = X.length().get([0])
    m = X.length().get([1])
    
    
    def centre(output):
        output = ee.Array(output)
        mean = output.reduce(ee.Reducer.mean(), [0]).get([0,0])
        return output.subtract(mean)
        
    def normalize(inputs):
        inputs = ee.Array(inputs)
        
        inputMeans = inputs.reduce(ee.Reducer.mean(), [0])
        inputMeans = inputMeans.repeat(0, n)
        inputs = inputs.subtract(inputMeans)
        inputs = inputs.pow(2).reduce(ee.Reducer.sum(), [0]).pow(-0.5).repeat(0,n).multiply(inputs)
        
        return inputs
    
    X = normalize(X)
    y = centre(y)
    
    def LARSregression(iteration, inputs):
        inputs = ee.Dictionary(inputs)
        prediction = inputs.getArray('prediction')
        coeff_arr = inputs.getArray('coeff_arr')
    
        c = X.matrixTranspose().matrixMultiply(y.subtract(prediction))
        c_abs = c.abs()
        C_max = c_abs.get(c_abs.argmax())

        maxLocs = c_abs.gte(C_max.subtract(0.00001))
        signs = c.divide(c_abs)

        signs_j = maxLocs.multiply(signs).matrixTranspose()
        signs_jc = signs_j.abs().subtract(1).multiply(-1)
        
        A = ee.List(ee.Array([ee.List.sequence(0, m.subtract(1))]).mask(signs_j).toList().get(0))
        A_c = ee.List(ee.Array([ee.List.sequence(0, m.subtract(1))]).mask(signs_jc).toList().get(0))

        signMatrix_j = signs_j.repeat(0, n)

        X_A = X.multiply(signMatrix_j).mask(signs_j)
        j = X_A.length().get([1])
        
        G_A = X_A.matrixTranspose().matrixMultiply(X_A)

        V1_A = ee.Array(ee.List.repeat([1], j))

        G_Ai = G_A.matrixInverse()

        A_A = V1_A.matrixTranspose().matrixMultiply(G_Ai).matrixMultiply(V1_A).get([0,0]).pow(-0.5)

        w_A = G_Ai.matrixMultiply(V1_A).multiply(A_A)

        u_A = X_A.matrixMultiply(w_A)

        a = X.matrixTranspose().matrixMultiply(u_A)

        def computeGammaRRay(index_j):
            minus_j = C_max.subtract(c.get([index_j, 0])).divide(A_A.subtract(a.get([index_j, 0])))
            plus_j = C_max.add(c.get([index_j, 0])).divide(A_A.add(a.get([index_j, 0])))

            gammaRRay = ee.Array([minus_j, plus_j]);
            gammaRRay = gammaRRay.mask(gammaRRay.gte(0))
            gammaRRay = gammaRRay.multiply(-1)

            return gammaRRay.get(gammaRRay.argmax())

        gammaRRay = ee.Array([A_c.map(computeGammaRRay)])
        gamma = gammaRRay.get(gammaRRay.argmax()).multiply(-1)

        prediction = prediction.add(u_A.multiply(gamma))
        coefficients = X.matrixSolve(prediction)

        def setZero(num):
            num = ee.Number(num)
            return ee.Algorithms.If(num.abs().lt(0.0001), [0], [num])

        coefficients = ee.Array(ee.List(coefficients.matrixTranspose().toList().get(0)).map(setZero))

        coeff_arr = ee.Array.cat([coeff_arr, coefficients], axis=1)

        outputs = ee.Dictionary({'prediction':prediction, 'coeff_arr':coeff_arr})

        return outputs
    
    
    numIterations = ee.List.sequence(1, n_nonzero)
    prediction = ee.Array(ee.List.repeat([0], n))
    coeff_arr = ee.Array(ee.List.repeat([0], m))
    initial = ee.Dictionary({'prediction':prediction, 'coeff_arr':coeff_arr})

    finalOutputs = numIterations.iterate(LARSregression, initial)
    finalOutputs = ee.Dictionary(finalOutputs)
    finalPrediction = finalOutputs.getArray('prediction')

    coeff_arr = finalOutputs.getArray('coeff_arr')
    coeff_arr = coeff_arr.getInfo()
    coeff_arr = numpy.asarray(coeff_arr)
    
    return coeff_arr

In [10]:
LAI_coef = EE_LARS_Regression('users/ccrs2fy2020/transformedFeatures', 
                              [ 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12',
                                'RVI1', 'RVI2', 'RVI3', 'RVI4', 'GM1', 'GM2', 'GI', 'II', 'MSI',
                                'PSSR', 'SGI', 'SR2', 'SR3', 'GVI', 'PSRI', 'NDVI3', 'SR5', 'SR6',
                                'SR7', 'IPVI', 'ARI', 'ARI2', 'NDVI', 'GNDVI', 'NDWI', 'NDREVI', 'NDGI',
                                'NDI1', 'NDI2', 'RENDVI', 'OSAVI', 'NMDI', 'HI', 'GVSP', 'MCARI',
                                'TCARI', 'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI', 'MSAVI2', 'MCARI2',
                                'MTVI2', 'N', 'GEMI', 'MSR2', 'NLI', 'TDVI' ], 
                              'LAI', 
                              50000, 
                              10)

In [11]:
FAPAR_coef = EE_LARS_Regression('users/ccrs2fy2020/transformedFeatures',
                                [ 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12',
                                'RVI1', 'RVI2', 'RVI3', 'RVI4', 'GM1', 'GM2', 'GI', 'II', 'MSI',
                                'PSSR', 'SGI', 'SR2', 'SR3', 'GVI', 'PSRI', 'NDVI3', 'SR5', 'SR6',
                                'SR7', 'IPVI', 'ARI', 'ARI2', 'NDVI', 'GNDVI', 'NDWI', 'NDREVI', 'NDGI',
                                'NDI1', 'NDI2', 'RENDVI', 'OSAVI', 'NMDI', 'HI', 'GVSP', 'MCARI',
                                'TCARI', 'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI', 'MSAVI2', 'MCARI2',
                                'MTVI2', 'N', 'GEMI', 'MSR2', 'NLI', 'TDVI' ],
                                'FAPAR', 
                                50000, 
                                10)

In [12]:
FCOVER_coef = EE_LARS_Regression('users/ccrs2fy2020/transformedFeatures', 
                                 [ 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12',
                                'RVI1', 'RVI2', 'RVI3', 'RVI4', 'GM1', 'GM2', 'GI', 'II', 'MSI',
                                'PSSR', 'SGI', 'SR2', 'SR3', 'GVI', 'PSRI', 'NDVI3', 'SR5', 'SR6',
                                'SR7', 'IPVI', 'ARI', 'ARI2', 'NDVI', 'GNDVI', 'NDWI', 'NDREVI', 'NDGI',
                                'NDI1', 'NDI2', 'RENDVI', 'OSAVI', 'NMDI', 'HI', 'GVSP', 'MCARI',
                                'TCARI', 'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI', 'MSAVI2', 'MCARI2',
                                'MTVI2', 'N', 'GEMI', 'MSR2', 'NLI', 'TDVI' ],
                                 'FCOVER', 
                                 50000, 
                                 10)

In [13]:
LAI_coef = pandas.DataFrame(LAI_coef)
FAPAR_coef = pandas.DataFrame(FAPAR_coef)
FCOVER_coef = pandas.DataFrame(FCOVER_coef)

In [29]:
inputs = outputCSV[['B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12',
                    'RVI1', 'RVI2', 'RVI3', 'RVI4', 'GM1', 'GM2', 'GI', 'II', 'MSI',
                    'PSSR', 'SGI', 'SR2', 'SR3', 'GVI', 'PSRI', 'NDVI3', 'SR5', 'SR6',
                    'SR7', 'IPVI', 'ARI', 'ARI2', 'NDVI', 'GNDVI', 'NDWI', 'NDREVI', 'NDGI',
                    'NDI1', 'NDI2', 'RENDVI', 'OSAVI', 'NMDI', 'HI', 'GVSP', 'MCARI',
                    'TCARI', 'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI', 'MSAVI2', 'MCARI2',
                    'MTVI2', 'N', 'GEMI', 'MSR2', 'NLI', 'TDVI']]

inputs_centred = inputs - inputs.mean()
inputs_normed = inputs_centred * inputs_centred.pow(2).sum().pow(-0.5)
n = inputs_normed.shape[0]
m = inputs_normed.shape[1]

In [30]:
numVars = 10
coefs = pandas.DataFrame(LAI_coef[numVars]).T
coefs.columns =['B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12',
                'RVI1', 'RVI2', 'RVI3', 'RVI4', 'GM1', 'GM2', 'GI', 'II', 'MSI',
                'PSSR', 'SGI', 'SR2', 'SR3', 'GVI', 'PSRI', 'NDVI3', 'SR5', 'SR6',
                'SR7', 'IPVI', 'ARI', 'ARI2', 'NDVI', 'GNDVI', 'NDWI', 'NDREVI', 'NDGI',
                'NDI1', 'NDI2', 'RENDVI', 'OSAVI', 'NMDI', 'HI', 'GVSP', 'MCARI',
                'TCARI', 'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI', 'MSAVI2', 'MCARI2',
                'MTVI2', 'N', 'GEMI', 'MSR2', 'NLI', 'TDVI']
coefs = coefs.transpose()
prediction = (inputs_normed.dot(coefs) + LAI_mean)[numVars]
RMSE = (((prediction - LAI).pow(2).sum())/n)**0.5
RMSE

0.8553778662089703

In [45]:
RMSE_min = numpy.Infinity
eps = 0.01*LAI_mean
for key, coefs in LAI_coef.items():
    predictions = inputs.dot(coefs) + LAI_mean
    RMSE = sklearn.metrics.mean_squared_error(y_true=LAI, y_pred=predictions, squared=False)
    if RMSE < RMSE_min+(0.01*LAI_mean)

B3 
 0        0.048953
1        0.053655
2        0.075767
3        0.071742
4        0.100043
           ...   
41467    0.091781
41468    0.046467
41469    0.045621
41470    0.053039
41471    0.077574
Name: B3, Length: 41472, dtype: float64
B4 
 0        0.030271
1        0.029552
2        0.092849
3        0.031115
4        0.028230
           ...   
41467    0.028950
41468    0.034942
41469    0.027923
41470    0.037354
41471    0.030683
Name: B4, Length: 41472, dtype: float64
B5 
 0        0.073742
1        0.074523
2        0.144840
3        0.094643
4        0.130998
           ...   
41467    0.124602
41468    0.069040
41469    0.067862
41470    0.081990
41471    0.106863
Name: B5, Length: 41472, dtype: float64
B6 
 0        0.294630
1        0.383288
2        0.267414
3        0.303518
4        0.442701
           ...   
41467    0.420813
41468    0.229278
41469    0.307144
41470    0.340293
41471    0.434154
Name: B6, Length: 41472, dtype: float64
B7 
 0        0.419698
1    

In [ ]:
LAI_coef = pandas.DataFrame(LAI_coef)
FAPAR_coef = pandas.DataFrame(FAPAR_coef)
FCOVER_coef = pandas.DataFrame(FCOVER_coef)

In [ ]:
LAI_features = numpy.nonzero(LAI_coef[5])[0]
FAPAR_features = numpy.nonzero(FAPAR_ceof[5])[0]
FCOVER_features = numpy.nonzero(FCOVER_coef[5])[0]

LAI_features = outputCSV.columns[LAI_features]
FAPAR_features = outputCSV.columns[FAPAR_features]
FCOVER_features = outputCSV.columns[FCOVER_features]

In [ ]:
LAI_model = tensorflow.keras.models.Sequential([
    tensorflow.keras.layers.Dense(10, activation=tensorflow.nn.sigmoid, 
                                  input_shape=[len(outputCSV[LAI_features].keys())]),
    tensorflow.keras.layers.Dense(10, activation=tensorflow.nn.sigmoid),
    tensorflow.keras.layers.Dense(1)
])
LAI_model.compile(
    optimizer=tensorflow.keras.optimizers.Nadam(),
    loss='mse',
    metrics=['mse', 'mae'])


FAPAR_model = tensorflow.keras.models.Sequential([
    tensorflow.keras.layers.Dense(10, activation=tensorflow.nn.sigmoid, 
                                  input_shape=[len(outputCSV[FAPAR_features].keys())]),
    tensorflow.keras.layers.Dense(10, activation=tensorflow.nn.sigmoid),
    tensorflow.keras.layers.Dense(1)
])
FAPAR_model.compile(
    optimizer=tensorflow.keras.optimizers.Nadam(),
    loss='mse',
    metrics=['mse', 'mae'])


FCOVER_model = tensorflow.keras.models.Sequential([
    tensorflow.keras.layers.Dense(10, activation=tensorflow.nn.sigmoid, 
                                  input_shape=[len(outputCSV[FCOVER_features].keys())]),
    tensorflow.keras.layers.Dense(10, activation=tensorflow.nn.sigmoid),
    tensorflow.keras.layers.Dense(1)
])

FCOVER_model.compile(
    optimizer=tensorflow.keras.optimizers.Nadam(),
    loss='mse',
    metrics=['mse', 'mae'])

In [ ]:
LAI_model.fit(x = inputDF[LAI_features], y = LAI, epochs = 100, validation_split = 0.2)
FAPAR_model.fit(x = inputDF[FAPAR_features], y = FAPAR, epochs = 20, validation_split = 0.2)
FCOVER_model.fit(x = inputDF[FCOVER_features], y = FCOVER, epochs = 20, validation_split = 0.2)

In [ ]:
LAI_predictions = pandas.Series(LAI_model.predict(inputDF[LAI_features]).flatten())
FAPAR_predictions = pandas.Series(FAPAR_model.predict(inputDF[FAPAR_features]).flatten())
FCOVER_predictions = pandas.Series(FCOVER_model.predict(inputDF[FCOVER_features]).flatten())

In [ ]:
xy_skl_LAI = numpy.vstack([LAI, LAI_predicted])
xy_tf_LAI = numpy.vstack([LAI, LAI_predictions])
xy_skl_FAPAR = numpy.vstack([FAPAR, FAPAR_predicted])
xy_tf_FAPAR = numpy.vstack([FAPAR, FAPAR_predictions])
xy_skl_FCOVER = numpy.vstack([FCOVER, FCOVER_predicted])
xy_tf_FCOVER = numpy.vstack([FCOVER, FCOVER_predictions])

In [ ]:
z_skl_LAI = scipy.stats.gaussian_kde(xy_skl_LAI)(xy_skl_LAI)
z_tf_LAI = scipy.stats.gaussian_kde(xy_tf_LAI)(xy_tf_LAI)
z_skl_FAPAR = scipy.stats.gaussian_kde(xy_skl_FAPAR)(xy_skl_FAPAR)
z_tf_FAPAR = scipy.stats.gaussian_kde(xy_tf_FAPAR)(xy_tf_FAPAR)
z_skl_FCOVER = scipy.stats.gaussian_kde(xy_skl_FCOVER)(xy_skl_FCOVER)
z_tf_FCOVER = scipy.stats.gaussian_kde(xy_tf_FCOVER)(xy_tf_FCOVER)

In [ ]:
idx_skl_LAI = z_skl_LAI.argsort()
idx_tf_LAI = z_tf_LAI.argsort()
idx_skl_FAPAR = z_skl_FAPAR.argsort()
idx_tf_FAPAR = z_tf_FAPAR.argsort()
idx_skl_FCOVER = z_skl_FCOVER.argsort()
idx_tf_FCOVER = z_tf_FCOVER.argsort()

In [ ]:
x_skl_LAI = LAI[idx_skl_LAI]
x_tf_LAI = LAI[idx_tf_LAI]
x_skl_FAPAR = FAPAR[idx_skl_FAPAR]
x_tf_FAPAR = FAPAR[idx_tf_FAPAR]
x_skl_FCOVER = FCOVER[idx_skl_FCOVER]
x_tf_FCOVER = FCOVER[idx_tf_FCOVER]

In [ ]:
y_skl_LAI = LAI_predicted[idx_skl_LAI]
y_tf_LAI = LAI_predictions[idx_tf_LAI]
y_skl_FAPAR = FAPAR_predicted[idx_skl_FAPAR]
y_tf_FAPAR = FAPAR_predictions[idx_tf_FAPAR]
y_skl_FCOVER = FCOVER_predicted[idx_skl_FCOVER]
y_tf_FCOVER = FCOVER_predictions[idx_tf_FCOVER]

In [ ]:
z_skl_LAI = z_skl_LAI[idx_skl_LAI]
z_tf_LAI = z_tf_LAI[idx_tf_LAI]
z_skl_FAPAR = z_skl_FAPAR[idx_skl_FAPAR]
z_tf_FAPAR = z_tf_FAPAR[idx_tf_FAPAR]
z_skl_FCOVER = z_skl_FCOVER[idx_skl_FCOVER]
z_tf_FCOVER = z_tf_FCOVER[idx_tf_FCOVER]

In [ ]:
rmse_skl_LAI = sklearn.metrics.mean_squared_error(x_skl_LAI, y_skl_LAI, squared=False)
rmse_tf_LAI = sklearn.metrics.mean_squared_error(x_tf_LAI, y_tf_LAI, squared=False)
rmse_skl_FAPAR = sklearn.metrics.mean_squared_error(x_skl_FAPAR, y_skl_FAPAR, squared=False)
rmse_tf_FAPAR = sklearn.metrics.mean_squared_error(x_tf_FAPAR, y_tf_FAPAR, squared=False)
rmse_skl_FCOVER = sklearn.metrics.mean_squared_error(x_skl_FCOVER, y_skl_FCOVER, squared=False)
rmse_tf_FCOVER = sklearn.metrics.mean_squared_error(x_tf_FCOVER, y_tf_FCOVER, squared=False)

In [ ]:
a_skl_LAI = numpy.linspace(0, 6, 1000)
a_tf_LAI = numpy.linspace(0, 10, 1000)
a_skl_FAPAR = numpy.linspace(0, 1, 1000)
a_tf_FAPAR = numpy.linspace(0, 1, 1000)
a_skl_FCOVER = numpy.linspace(0, 1, 1000)
a_tf_FCOVER = numpy.linspace(0, 1, 1000)

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(25,15))


ax[0,0].scatter(x_skl_LAI, y_skl_LAI, c=z_skl_LAI)
ax[0,0].plot(a_skl_LAI, a_skl_LAI, c='r')
ax[0,0].set_title('LASSO LARS LAI - RMSE: {}'.format(rmse_skl_LAI))


ax[1,0].scatter(x_tf_LAI, y_tf_LAI, c=z_tf_LAI)
ax[1,0].plot(a_tf_LAI, a_tf_LAI, c='r')
ax[1,0].set_title('LASSO LARS FAPAR - RMSE: {}'.format(rmse_tf_LAI))


ax[0,1].scatter(x_skl_FAPAR, y_skl_FAPAR, c=z_skl_FAPAR)
ax[0,1].plot(a_skl_FAPAR, a_skl_FAPAR, c='r')
ax[0,1].set_title('LASSO LARS FCOVER - RMSE: {}'.format(rmse_skl_FAPAR))


ax[1,1].scatter(x_tf_FAPAR, y_tf_FAPAR, c=z_tf_FAPAR)
ax[1,1].plot(a_tf_FAPAR, a_tf_FAPAR, c='r')
ax[1,1].set_title('NNet LAI - RMSE: {}'.format(rmse_tf_FAPAR))


ax[0,2].scatter(x_skl_FCOVER, y_skl_FCOVER, c=z_skl_FCOVER)
ax[0,2].plot(a_skl_FCOVER, a_skl_FCOVER, c='r')
ax[0,2].set_title('NNet FAPAR - RMSE: {}'.format(rmse_skl_FCOVER))


ax[1,2].scatter(x_tf_FCOVER, y_tf_FCOVER, c=z_tf_FCOVER)
ax[1,2].plot(a_tf_FCOVER, a_tf_FCOVER, c='r')
ax[1,2].set_title('NNet FCOVER - RMSE: {}'.format(rmse_tf_FCOVER))